In [2]:
import google.generativeai as genai
import os
import seaborn as sns
import pandas as pd

c:\python_\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Auth**

In [13]:
import google.generativeai as genai
for i, m in zip(range(4), genai.list_tuned_models()):
  print(m.name)


tunedModels/domain4cookingggg


In [ ]:
genai.delete_tuned_model("tunedModels/domain4cooking")

# **Dominio - Cocina**

## **Ajuste de Modelo**

In [14]:
base_model = [
    m for m in genai.list_models()
    if "createTunedModel" in m.supported_generation_methods][0]
base_model

Model(name='models/gemini-1.0-pro-001',
      base_model_id='',
      version='001',
      display_name='Gemini 1.0 Pro 001 (Tuning)',
      description=('The best model for scaling across a wide range of tasks. This is a stable '
                   'model that supports tuning.'),
      input_token_limit=30720,
      output_token_limit=2048,
      supported_generation_methods=['generateContent', 'countTokens', 'createTunedModel'],
      temperature=0.9,
      max_temperature=None,
      top_p=1.0,
      top_k=None)

## Datos de entrenamiento

In [15]:
training_cooking = [
    # Introducción del rol que jugará el modelo
    {"text_input": "Quiero que juegues el siguiente rol: Eres un asistente especializado en reconocer ingredientes para comidas y sugerir recetas a partir de ellos. Además, continuarás la conversación para guiar al usuario en la preparación de la receta y sugerirás la compra de ingredientes faltantes, esperando su respuesta.",
     "output": "Entendido. Soy un asistente especializado en sugerir recetas basadas en los ingredientes que el usuario tiene y guiarlo paso a paso. También puedo sugerir ingredientes adicionales y preguntar si desea comprarlos."},

    # Variación - Usuario con restricciones alimenticias
    {"text_input": "Quiero que juegues el siguiente rol: Eres un asistente que reconoce ingredientes y sugiere recetas para personas con intolerancia al gluten. Me guiarás en la preparación y sugerirás alternativas si me faltan ingredientes.",
     "output": "Entendido. Soy un asistente especializado en sugerir recetas sin gluten basadas en los ingredientes que tienes. Te guiaré en cada paso y te sugeriré alternativas si es necesario."},

    # Conversación sobre una receta con ingredientes disponibles
    {"text_input": "Tengo tomates, pasta y ajo.", "output": "Con esos ingredientes, podrías hacer una pasta al pomodoro. ¿Te gustaría que te guíe en la preparación?"},
    {"text_input": "Sí, quiero empezar.", "output": "Primero, sofríe el ajo con aceite de oliva. ¿Tienes aceite de oliva?"},
    {"text_input": "No, no tengo.", "output": "No hay problema, puedes usar aceite vegetal. ¿Te gustaría que te sugiera comprar aceite de oliva para otras recetas?"},
    {"text_input": "Sí, sugiereme comprar.", "output": "Te recomiendo comprar aceite de oliva, ya que es ideal para muchas recetas. Ahora, añade los tomates picados. ¿Ya lo tienes listo?"},

    # Conversación sobre una ensalada
    {"text_input": "Tengo lechuga, tomate y zanahoria. ¿Qué puedo hacer?", "output": "Puedes hacer una ensalada fresca. ¿Te gustaría que te guíe en la preparación?"},
    {"text_input": "Sí, por favor.", "output": "Lava y corta la lechuga, el tomate y la zanahoria. ¿Tienes aderezo para ensaladas?"},
    {"text_input": "No, no tengo.", "output": "Puedes hacer un aderezo sencillo con aceite de oliva, sal y limón. ¿Te gustaría comprar un aderezo listo?"},
    {"text_input": "Sí, lo haré.", "output": "Genial. Mezcla los ingredientes y añade el aderezo. ¡Tu ensalada está lista!"},

    # Conversación sobre alternativas de recetas
    {"text_input": "Tengo harina, levadura y queso. ¿Puedo hacer pizza?", "output": "Con esos ingredientes, puedes hacer una masa para pizza. ¿Te gustaría que te ayude a preparar una pizza sencilla?"},
    {"text_input": "Sí, quiero hacer pizza.", "output": "Mezcla la harina, levadura y agua. Amasa y deja reposar. ¿Tienes tomate para la salsa?"},
    {"text_input": "No, no tengo tomate.", "output": "Podrías comprar tomates para hacer una salsa fresca. ¿Te gustaría hacerlo o prefieres usar otra base?"},

    # Conversación sobre recetas rápidas
    {"text_input": "Quiero hacer una comida rápida con arroz y pollo.", "output": "Con esos ingredientes, podrías hacer un arroz con pollo sencillo. ¿Te gustaría que te guíe en la preparación?"},
    {"text_input": "Sí, quiero hacerlo rápido.", "output": "Primero, cocina el pollo con un poco de sal. ¿Ya lo tienes listo?"},
    {"text_input": "Sí, el pollo está cocido.", "output": "Perfecto, añade el arroz al pollo y un poco de agua. Cocina a fuego medio por 15 minutos. ¿Te gustaría agregar alguna verdura?"},

    # Conversación sobre recetas saludables
    {"text_input": "Quiero hacer algo saludable con plátano y avena.", "output": "Puedes hacer galletas saludables con esos ingredientes. ¿Te gustaría que te guíe en la receta?"},
    {"text_input": "Sí, por favor.", "output": "Mezcla los plátanos machacados con la avena. ¿Tienes canela para darle más sabor?"},
    {"text_input": "No, no tengo canela.", "output": "Podrías comprar canela para futuras recetas. ¿Te gustaría hacerlo?"},

    # Conversación sobre un postre con chocolate
    {"text_input": "Tengo chocolate y crema de leche. ¿Qué puedo hacer?", "output": "Puedes hacer una mousse de chocolate. ¿Te gustaría que te guíe en la preparación?"},
    {"text_input": "Sí, me gustaría.", "output": "Derrite el chocolate a baño maría. ¿Lo tienes listo?"},
    {"text_input": "Sí, el chocolate está derretido.", "output": "Ahora, mezcla el chocolate derretido con la crema de leche batida. Deja enfriar en la nevera por 2 horas. ¡Tu mousse estará lista!"},

    # Conversación sobre ingredientes faltantes
    {"text_input": "Tengo huevos, azúcar y harina. ¿Qué puedo hacer?", "output": "Con esos ingredientes, podrías hacer un bizcocho básico. ¿Te gustaría que te guíe?"},
    {"text_input": "Sí, pero no tengo polvo de hornear.", "output": "Puedes comprar polvo de hornear para darle más volumen al bizcocho. ¿Te gustaría comprarlo?"},
    {"text_input": "No, prefiero no comprar ahora.", "output": "No hay problema. Puedes hacer el bizcocho sin polvo de hornear, pero será un poco más denso."},

    # Variación - Usuario que busca recetas vegetarianas
    {"text_input": "Quiero que juegues el siguiente rol: Eres un asistente que sugiere recetas vegetarianas a partir de los ingredientes que tengo disponibles. Además, continuarás la conversación para guiarme en la preparación y sugerirás la compra de ingredientes adicionales si es necesario.",
     "output": "Entendido. Soy un asistente especializado en sugerir recetas vegetarianas basadas en los ingredientes que tienes. Te guiaré en cada paso de la preparación y sugeriré ingredientes adicionales si los necesitas."},
]



In [16]:
name = f'domain4cookingggg'
operation = genai.create_tuned_model(
    # You can use a tuned model here too. Set `source_model="tunedModels/..."`
    source_model=base_model.name,
 training_data = training_cooking,
    id = name,
    epoch_count = 50,
    batch_size=4,
    learning_rate=0.001,
)

AlreadyExists: 409 Tuned model domain4cookingggg already exists.

In [17]:
model = genai.get_tuned_model(f'tunedModels/{name}')

model

TunedModel(name='tunedModels/domain4cookingggg',
           source_model='models/gemini-1.0-pro-001',
           base_model='models/gemini-1.0-pro-001',
           display_name='',
           description='',
           temperature=0.9,
           top_p=1.0,
           top_k=0,
           state=<State.ACTIVE: 2>,
           create_time=datetime.datetime(2024, 10, 2, 22, 44, 42, 682924, tzinfo=datetime.timezone.utc),
           update_time=datetime.datetime(2024, 10, 2, 22, 49, 47, 985515, tzinfo=datetime.timezone.utc),
           tuning_task=TuningTask(start_time=datetime.datetime(2024, 10, 2, 22, 44, 43, 514960, tzinfo=datetime.timezone.utc),
                                  complete_time=datetime.datetime(2024, 10, 2, 22, 49, 47, 985515, tzinfo=datetime.timezone.utc),
                                  snapshots=[...],
                                  hyperparameters=Hyperparameters(epoch_count=50,
                                                                  batch_size=4,
      

In [9]:
import time

for status in operation.wait_bar():
  time.sleep(30)

  0%|          | 0/325 [00:30<?, ?it/s]


KeyboardInterrupt: 

In [12]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')

NameError: name 'operation' is not defined

In [18]:
model = genai.GenerativeModel(model_name=f'tunedModels/{name}')

In [19]:
result = model.generate_content('Me gustaria cocinar algo para mi, pero no sé qué hacer')
result.text

'**Platos sencillos para principiantes:**\n\n* **Pasta con tomate:** Cocina pasta según las instrucciones del paquete. Sofríe ajo con aceite de oliva y añade tomates triturados. Cocina a fuego lento por 15 minutos.\n* **Arroz con pollo:**Cocina arroz con agua. Sofríe pollo con aceite de oliva y añade verduras como zanahoria y arveja. Mezcla con el arroz cocido.\n* **Tortilla de patatas:** Bate huevos con sal. Sofríe patatas cortadas en cubos. Añade los huevos batidos y cocina a fuego medio.\n\n**Recetas más elaboradas:**\n\n* **Pollo al horno con verduras:** Hornea pollo con verduras como papas, zanahorias y cebollas. Sazona con sal, pimienta y aceite de oliva.\n* **Lasaña de carne:** Cocina pasta para lasaña. Prepara una salsa de tomate y una boloñesa de carne. Arma la lasaña intercalando pasta, salsa y queso. Hornea por 30 minutos.\n* **Pizza casera:** Amasa harina, levadura y agua. Deja reposar. Extiende la masa y añade salsa de tomate, queso y tus ingredientes favoritos. Hornea a a

In [ ]:
!pip install speechrecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 29.3 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install portaudio19-dev python3-pyaudio

!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc python-pyaudio-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev python3-pyaudio
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 213 kB of archives.
After this operation, 1,043 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pyaudio amd64 0.2.11-1.3ubuntu1 [25.9 kB]
Fetched 213 kB in 0s (636 kB/s)
debconf: unable to initialize fron

## **Chat con modelo e imágenes**

In [22]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
from PIL import Image, ImageTk
import speech_recognition as sr
import google.generativeai as genai
import cv2
import numpy as np
import threading
import requests
import pygame
import os

# Inicializa el reconocimiento de voz
reconocedor = sr.Recognizer()

# Configura el modelo generativo
model_img = genai.GenerativeModel("gemini-1.5-flash")
name = "domain4cookingggg"
model = genai.GenerativeModel(model_name=f'tunedModels/{name}')
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "el modelo debe actuar como un profesor de culinaria. Recibe una lista de ingredientes y debe proporcionarle al usuario una lista de pasos y guiar al usuario para que efectúe la receta. Solo puede sugerir recetas con los ingredientes que recibe en la lista, únicamente esos."},
        {"role": "model", "parts": "Bien. Dime los ingredientes, y te sugiriré ingrientes, y te daré los pasos, de acuerdo a ellos. Solo los ingredientes que me digas"},
    ]
)

recording = False
audio_thread = None

API_KEY = 'sk_7177886669223a38d387fbd8d17d98faefd6ca59c3146d97'  # Reemplaza con tu API Key de Eleven Labs


def sintetizar_voz(texto):
    # Directorio temporal para guardar el archivo de audio
    temp_dir = os.path.join(os.path.expanduser("~"), "AppData", "Local", "Temp")
    audio_path = os.path.join(temp_dir, "respuesta_audio.mp3")

    # Si el archivo ya existe, elimínalo antes de escribir uno nuevo
    if os.path.exists(audio_path):
        os.remove(audio_path)

    url = "https://api.elevenlabs.io/v1/text-to-speech/9BWtsMINqrJLrRacOk9x"  # Cambia YOUR_VOICE_ID por el ID de la voz que quieras usar
    headers = {
        'accept': 'audio/mpeg',
        'xi-api-key': API_KEY,  # Asegúrate de reemplazar con tu API_KEY
        'Content-Type': 'application/json',
    }
    data = {
        "text": texto,
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.5
        }
    }

    # Solicitud a la API para sintetizar la voz
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        # Guardar el archivo de audio en el directorio temporal
        with open(audio_path, "wb") as f:
            f.write(response.content)
        return audio_path
    else:
        return None


def reproducir_audio(file):
    pygame.mixer.init()  # Inicializa el mezclador de audio
    pygame.mixer.music.load(file)  # Carga el archivo MP3
    pygame.mixer.music.play()  # Reproduce el archivo

    # Mantén el programa en ejecución hasta que el archivo de audio termine
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)  # Pausa el bucle para evitar bloqueo

    pygame.mixer.music.stop()  # Detener la reproducción
    pygame.mixer.quit()  # Cierra el mixer y libera el archivo de audio


def reproducir_audio_thread(file):
    thread = threading.Thread(target=reproducir_audio, args=(file,))
    thread.start()


def reconocer_voz():
    global recording
    if not recording:
        return

    with sr.Microphone() as source:
        reconocedor.energy_threshold = noise_slider.get()
        reconocedor.adjust_for_ambient_noise(source, duration=1)
        audio = reconocedor.listen(source)

        try:
            texto = reconocedor.recognize_google(audio, language="es-ES")
            return texto
        except sr.UnknownValueError:
            return "Could not understand the audio"
        except sr.RequestError as e:
            return f"Error connecting to the speech recognition service; {e}"


def load_image():
    file_path = filedialog.askopenfilename(
        filetypes=[("Image files", ".jpg;.jpeg;.png;.bmp;*.gif")],
        title="Select an image"
    )

    if file_path:
        image = Image.open(file_path)
        return image
    else:
        return None


def capture_from_camera():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        return None
    
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame_rgb)
        return image
    else:
        return None


def send_message():
    texto = input_text.get("1.0", "end-1c")
    if not texto:
        return

    response = chat.send_message(
        texto,
        generation_config=genai.types.GenerationConfig(
            candidate_count=1,
            stop_sequences=["x"],
            max_output_tokens=50,
            temperature=0.7
        )
    )
    respuesta_texto = response.text

    chat_history.insert(tk.END, f"You: {texto}\n")
    chat_history.insert(tk.END, f"Model: {respuesta_texto}\n")
    input_text.delete("1.0", tk.END)

    # Generar y reproducir el audio con Eleven Labs
    archivo_audio = sintetizar_voz(respuesta_texto)
    if archivo_audio:
        reproducir_audio_thread(archivo_audio)


def handle_audio():
    global audio_thread
    audio_thread = threading.Thread(target=lambda: process_audio())
    audio_thread.start()


def process_audio():
    global recording
    recording = True
    texto = reconocer_voz()
    if texto:
        input_text.delete("1.0", tk.END)
        input_text.insert(tk.END, texto)
        send_message()
    recording = False


def handle_image(image):
    if image:
        try:
            response = model_img.generate_content(
                ["Act as a culinary master and identify each ingredient you see in the image in detail. Be concise and just print the ingredient list. Do something like: The ingredients: (and insert the list)", image],
                generation_config=genai.types.GenerationConfig(
                    candidate_count=1,
                    stop_sequences=["x"],
                    max_output_tokens=50,
                    temperature=0.7
                )
            )
            chat_history.insert(tk.END, f"You: Image loaded\n")
            chat_history.insert(tk.END, f"Model: {response.text}\n")
        except Exception as e:
            chat_history.insert(tk.END, f"Error processing the image: {e}\n")


def on_send_click():
    send_message()


def on_audio_button_press(event):
    handle_audio()


def on_audio_button_release(event):
    global recording
    recording = False


def on_image_click():
    image = load_image()
    if image:
        handle_image(image)
        img_tk = ImageTk.PhotoImage(image)
        image_label.config(image=img_tk)
        image_label.image = img_tk


def on_camera_click():
    image = capture_from_camera()
    if image:
        handle_image(image)
        img_tk = ImageTk.PhotoImage(image)
        camera_label.config(image=img_tk)
        camera_label.image = img_tk


def on_clear_click():
    chat_history.delete(1.0, tk.END)
    image_label.config(image='')
    camera_label.config(image='')


# Configura la ventana de la interfaz gráfica
root = tk.Tk()
root.title("Chat with Culinary Model")

main_frame = tk.Frame(root, padx=10, pady=10)
main_frame.pack(expand=True, fill=tk.BOTH)

left_frame = tk.Frame(main_frame, padx=10, pady=10)
left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

right_frame = tk.Frame(main_frame, padx=10, pady=10)
right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

title_label = tk.Label(right_frame, text="Chat with Culinary Model", font=("Helvetica", 16, "bold"))
title_label.pack(pady=5)

chat_history = scrolledtext.ScrolledText(right_frame, wrap=tk.WORD, height=15, width=40)
chat_history.pack(pady=10, fill=tk.BOTH, expand=True)

input_text = tk.Text(right_frame, height=3, width=40)
input_text.pack(pady=5, fill=tk.BOTH, expand=True)

slider_frame = tk.Frame(right_frame, pady=10)
slider_frame.pack(fill=tk.X)

slider_label = tk.Label(slider_frame, text="Noise Sensitivity:")
slider_label.pack(side=tk.LEFT, padx=5)

noise_slider = tk.Scale(slider_frame, from_=100, to=10000, orient=tk.HORIZONTAL)
noise_slider.set(reconocedor.energy_threshold)
noise_slider.pack(side=tk.LEFT, padx=5, fill=tk.X, expand=True)

button_frame = tk.Frame(right_frame)
button_frame.pack(pady=10)

send_button = tk.Button(button_frame, text="Send", command=on_send_click, width=15)
send_button.grid(row=0, column=0, padx=5)

audio_button = tk.Button(button_frame, text="Voice Recognition", width=20)
audio_button.grid(row=0, column=1, padx=5)
audio_button.bind("<ButtonPress>", on_audio_button_press)
audio_button.bind("<ButtonRelease>", on_audio_button_release)

image_button = tk.Button(button_frame, text="Load Image", command=on_image_click, width=15)
image_button.grid(row=0, column=2, padx=5)

camera_button = tk.Button(button_frame, text="Capture Camera", command=on_camera_click, width=15)
camera_button.grid(row=0, column=3, padx=5)

clear_button = tk.Button(button_frame, text="Clear", command=on_clear_click, width=15)
clear_button.grid(row=0, column=4, padx=5)

image_label = tk.Label(left_frame)
image_label.pack(pady=10)

camera_label = tk.Label(left_frame)
camera_label.pack(pady=10)

# Inicia el bucle principal
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
chat.history

[parts {
   text: "Estoy a dieta porque quiero bajar de peso."
 }
 role: "user"]

## sugerencias con urls


In [ ]:
from PIL import Image
from google.colab import files
import base64
import IPython.display as display  # Para mostrar la imagen en Colab

# Inicializar el modelo de Gemini
model_img = genai.GenerativeModel("gemini-1.5-flash")

# Cargar el modelo ajustado
name = "domain4cooking"
model = genai.GenerativeModel(model_name=f'tunedModels/{name}')

# Iniciar el chat con el modelo ajustado
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Estoy a dieta porque quiero bajar de peso."},
    ]
)

# Bucle de conversación
while True:
    text = input("Digita una pregunta (escribe 'imagen' para cargar una imagen o 'salir' para terminar): ")

    # Si el usuario escribe 'salir', finaliza el ciclo
    if text.lower() == "salir":
        print("¡Hasta luego!")
        break

    # Opción para cargar una imagen
    if text.lower() == "imagen":
        print("Por favor, carga una imagen.")

        # Permitir al usuario cargar un archivo
        uploaded = files.upload()

        # Obtener la ruta del archivo cargado
        image_path = list(uploaded.keys())[0]

        try:
            # Cargar la imagen usando PIL
            image = Image.open(image_path)
            print("Imagen cargada correctamente.")

            # Mostrar la imagen directamente en Colab
            display.display(image)

            # Enviar la imagen (en base64) y el texto al modelo
            response = model_img.generate_content(
                ["Actúa como un maestro culinario e identifica detalladamente cada ingrediente que ves en la imagen. Ve al punto y solo muestra la lista de ingredientes. Haz algo como: Los ingredientes son: (e inserta la lista)", image],
                generation_config=genai.types.GenerationConfig(
                    candidate_count=1,        # Una sola respuesta
                    stop_sequences=["x"],     # Secuencia opcional de parada
                    max_output_tokens=50,     # Máximo de 50 tokens
                    temperature=0.7           # Ajusta la aleatoriedad
                )
            )
            # Mostrar la lista de ingredientes identificados
            print(response.text)

            # Enviar mensaje con los ingredientes al modelo ajustado
            response = chat.send_message(
                f"Tengo estos ingredientes: {response.text}. Proporciona un enlace a una receta en línea basada en ellos.",
                generation_config=genai.types.GenerationConfig(
                    candidate_count=1,        # Una sola respuesta
                    stop_sequences=["x"],     # Secuencia opcional de parada
                    max_output_tokens=150,    # Mayor cantidad de tokens para respuestas más largas
                    temperature=0.7           # Ajusta la aleatoriedad
                )
            )

            # Mostrar la respuesta del modelo con el link a la receta
            print(response.text)

        except Exception as e:
            print(f"No se pudo cargar la imagen. Error: {e}")
        continue

    # Enviar mensaje normal con el texto ingresado
    response = chat.send_message(
        f"{text} Proporciona un enlace a una receta en línea relacionada con los ingredientes que mencionas.",
        generation_config=genai.types.GenerationConfig(
            candidate_count=1,        # Una sola respuesta
            stop_sequences=["x"],     # Secuencia opcional de parada
            max_output_tokens=150,    # Máximo de 150 tokens para permitir descripciones más largas
            temperature=0.7           # Ajusta la aleatoriedad
        )
    )

    # Mostrar la respuesta del modelo con el link a la receta
    print(response.text)


No tengo acceso a internet para proporcionar un enlace a una receta.


In [24]:
pip install google-cloud-aiplatform


   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   --------------------------------- ------ 4.5/5.3 MB 26.7 MB/s eta 0:00:01
   ---------------------------------------- 5.3/5.3 MB 20.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 25.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [26]:
import google.auth
from google.cloud import aiplatform_v1

# Autenticación
credentials, project_id = google.auth.default()

# Cliente para el servicio de AI Platform
client = aiplatform_v1.ModelServiceClient()

# Verifica los modelos en tu proyecto y región
parent = f"projects/{project_id}/locations/us-central1"
models = client.list_models(parent=parent)

# Lista los modelos disponibles
for model in models:
    print(model.name)


MethodNotImplemented: 501 Received http2 header with status: 404